In [1]:
import pickle
from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_runtime import Sampler, Options, QiskitRuntimeService
import qiskit_dynamics
import numpy as np
from hamiltonian_models import Ising
from qiskit_ibm_provider import IBMProvider
from simuq.solver import generate_as

In [2]:
api_file = "../../ibm_API_key"
with open(api_file, "r") as f:
    api_key = f.readline().strip()
provider = IBMProvider(api_key, instance='ibm-q-ncsu/nc-state/quantum-compiler')

In [3]:
N = 4
T = 1
system = 'ibm_brisbane'

In [4]:
h = np.array([1 for j in range(N)])
J_chain = np.zeros((N, N))
for j in range(N - 1):
    J_chain[j, j + 1] = 1

J_cycle = np.copy(J_chain)
J_cycle[0, N - 1] = 1

Ising_chain = Ising(N, T, J_chain, h)

In [8]:
def compile(
        qs,
        backend="ibmq_mumbai",
        aais="heisenberg",
        tol=0.01,
        trotter_num=6,
        verbose=0,
        use_pulse=True,
        state_prep=None,
        use_fake_backend=False,
        with_measure=True
    ):

    #SimuQ code
    backend = provider.get_backend(backend)

    nsite = backend.configuration().n_qubits

    if qs.num_sites > nsite:
        raise Exception("Device has less sites than the target quantum system.")

    if aais == "heisenberg":
        import ibm
        from qiskit_pulse_ibm import transpile

        mach = ibm.generate_qmachine(backend)
        comp = transpile

    layout, sol_gvars, boxes, edges = generate_as(
        qs,
        mach,
        trotter_num,
        solver="least_squares",
        solver_args={"tol": tol},
        override_layout=None,
        verbose=verbose,
    )

    prog = []

    from qiskit import transpile as transpile_qiskit

    for noise_factor in range(0,5):

        prog_circ = comp(
            backend,
            layout,
            sol_gvars,
            boxes,
            edges,
            use_pulse=use_pulse,
            with_measure=with_measure,
            noise_factor=noise_factor,
            circuit_length=qs.num_sites
        )

        prog_circ = transpile_qiskit(prog_circ, backend=backend)

        if state_prep is not None:
            prog_circ = prog_circ.compose(state_prep, qubits=layout, front=True)

        prog.append(prog_circ)
    
    return prog, layout

In [9]:
circuit, layout = compile(Ising_chain,backend=system)

TypeError: transpile() got an unexpected keyword argument 'circuit_length'

In [7]:
circuit[0].draw(output='mpl')

ValueError: Image size of 2336x97022 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 2336.56x97022.3 with 1 Axes>

### Method 1

In [6]:
backend = provider.get_backend('ibm_sherbrooke')
backend_sim = qiskit_dynamics.DynamicsBackend.from_backend(backend, subsystem_list=[0,1,2,3])

In [10]:
from qiskit import transpile

circ = transpile(circuit, backend_sim)
job = backend_sim.run(circuit, shots=4096, seed_simulator=12345)

TranspilerError: "Unable to translate the operations in the circuit: ['sx', 'measure', 'barrier', 'rz'] to the backend's (or manually specified) target basis: ['barrier', 'snapshot', 'measure']. This likely means the target basis is not universal or there are additional equivalence rules needed in the EquivalenceLibrary being used. For more details on this error see: https://qiskit.org/documentation/stubs/qiskit.transpiler.passes.BasisTranslator.html#translation_errors"

### Method 2

In [16]:
len(circuit)

5

In [15]:
circuit[0]

In [17]:
circ

NameError: name 'circ' is not defined

In [18]:
#Get the backend
backend = provider.get_backend('ibm_brisbane')

# transpile to backend
circ = transpile(circuit, backend=backend)

In [19]:
circ

### Method 3

In [21]:
import qiskit

qiskit.__qiskit_version__

{'qiskit': '0.45.0', 'qiskit-aer': '0.11.0', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': None, 'qiskit-nature': '0.5.2', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [9]:
from qiskit_aer.noise import NoiseModel
from qiskit import Aer

noisy_backend = provider.get_backend('ibm_brisbane')
simulator = provider.get_backend('ibmq_qasm_simulator')
noise_model = NoiseModel.from_backend(noisy_backend).to_dict()
simulator.options.update_options(noise_model=noise_model)

job2 = execute(circuit, shots=4096, backend=simulator)

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\qiskit_aer\noise\device\models.py:145: UserWarning: When `target` is supplied, `standard_gates` and `warnings` are ignored, and they are always set to true.
  warn("When `target` is supplied, `standard_gates` and `warnings` are ignored,"
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\qiskit_aer\noise\device\models.py:365: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (0.00013013589288602713 > 2 * 5.4487918644715075e-05). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\qiskit_aer\noise\device\models.py:365: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (0.00013013589288602713 > 2 * 5.4487918644715075e-05). Truncating to maximum value.
  warn("Device model ret

In [6]:
job.status()

<JobStatus.RUNNING: 'job is actively running'>

In [14]:
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import Session, Sampler

service = QiskitRuntimeService(channel="ibm_quantum")
noisy_backend = service.get_backend('ibm_brisbane')
backend_noise_model = NoiseModel.from_backend(noisy_backend)

simulator = service.get_backend('ibmq_qasm_simulator')

options = Options()
options.resilience_level = 0
options.optimization_level = 0
options.simulator = {
    "noise_model": backend_noise_model
}

with Session(service=service, backend=simulator) as session:
    sampler = Sampler(session=session, options=options)
    job = sampler.run(circuits=circuit)
    print(job.result().values)

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\qiskit_aer\noise\device\models.py:145: UserWarning: When `target` is supplied, `standard_gates` and `warnings` are ignored, and they are always set to true.
  warn("When `target` is supplied, `standard_gates` and `warnings` are ignored,"


: 

In [9]:
from qiskit_aer.noise import NoiseModel

noise_model = NoiseModel.from_backend(FakeMumbaiV2()).to_dict()
simulator = provider.get_backend("ibmq_qasm_simulator")
simulator.options.update_options(noise_model=noise_model)

job = execute(circuit, shots=4096, backend=simulator)

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\qiskit_aer\noise\device\models.py:145: UserWarning: When `target` is supplied, `standard_gates` and `warnings` are ignored, and they are always set to true.
  warn("When `target` is supplied, `standard_gates` and `warnings` are ignored,"


In [11]:
job.status()

<JobStatus.ERROR: 'job incurred error'>

In [ ]:
job_params = []

for N in range(4,11):
    for T in range(1,3):
        for system in ['ibm_brisbane','ibm_kyoto','ibm_sherbrooke','ibmq_mumbai','ibm_nazca']:
            filename = "circuits/" + system + "_" + str(N) + "_" + str(T) + ".pkl"
            with open(filename, 'rb') as handle:
                circuit = pickle.load(handle)['circuit']
            
            #Get the backend
            #backend = provider.get_backend(system)
            #job = execute(circuit, shots=4096, backend=backend)

            #Execute using Sampler
            backend = service.backend(system)
            options = Options()
            options.resilience_level = 1
            sampler = Sampler(backend=backend,options=options)

            job = sampler.run(circuit, shots=4096)

            #Append the job parameters
            job_params.append({'system': system, 'N': N, 'T': T, 'job_id': job.job_id()})
            print(job_params)

#Save the job parameters
with open("circuits/job_params_sampler.pkl", "wb") as f:
    pickle.dump(job_params, f)